In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(r'/tf/Lecture/Covid_19_Prediction_Stacking/src/')

# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import numpy as np

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader



from my_dataloader import My_DataLoader
from lstm import *





In [ ]:
lr = 0.00002
epochs = 1000
verbose = True
patients = 10
verify_every = 5
use_cuda = True

In [ ]:
dl = My_DataLoader()
dl.load_from_csv()
country = 'Austria'
ori_data = dl.countries_data[country]
ori_norm_data = dl.norm_countries_data[country]
valid_start = ori_norm_data[ori_norm_data['date']=='11/10/20'].index.values[0]
valid_end = ori_norm_data[ori_norm_data['date']=='12/10/20'].index.values[0]
ori_norm_data = ori_norm_data.drop(columns=['date'])

index_map = {}
for i, col in enumerate(ori_data.columns.values):
    index_map[col] = i

In [ ]:
index_map


In [ ]:
x_window_train = []
for i in range(dl.time_steps, valid_start):
    # print('i', i)
    new_x = []
    for j in range(i - dl.time_steps, i):
        # print('j', j)
        row = ori_norm_data.loc[j].values.tolist()
        new_x.append(row)
        
    x_window_train.append(new_x)
    # print(df.shape)
    
y_train = ori_data[dl.time_steps : valid_start][['total_cases', 'total_deaths', 'total_recovered', 'total_tests']]
x_valid = ori_norm_data[valid_start - dl.time_steps + 1 : valid_start + 1].to_numpy()
y_valid = ori_data[valid_start : valid_end][['total_cases', 'total_deaths', 'total_recovered', 'total_tests']]

In [ ]:
train_data = TensorDataset(torch.from_numpy(np.array(x_window_train)).float(), torch.from_numpy(y_train.values.astype('float')).float())
train_loader = DataLoader(train_data, shuffle=False, batch_size=dl.batch_size)


#
# train_data = TensorDataset(torch.from_numpy(x_train.values), torch.from_numpy(y_train.values))
# valid_data = TensorDataset(torch.from_numpy(x_valid.values), torch.from_numpy(y_valid.values))
# test_data = TensorDataset(torch.from_numpy(data_loader.x_test.values), torch.from_numpy(data_loader.y_test.values))
# train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size, drop_last=True)
# valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size, drop_last=True)
# test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)
#
# data = data_loader.data
# feature_sizes = []
# for col in data.columns:
#     if col in data_loader.continuous_column:
#         feature_sizes.append(1)
#     elif col == 'is_trade':
#         continue
#     else:
#         feature_sizes.append(len(data[col].unique()) + 50)
#
# net = DeepFM(feature_sizes=feature_sizes, continuous_field_size=len(data_loader.continuous_column), num_classes=2,
#              use_cuda=use_cuda)
# optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=0.03)
#
# if use_cuda:
#     net.cuda()
# model = net.fit(train_loader, valid_loader, optimizer, epochs=epochs, verbose=verbose, print_every=print_every)
#
# test_loss = net.validation(test_loader, model)
# print('test_log_loss: {0:f}'.format(test_loss))
#

In [ ]:
config = LSTM_Config(feature_size=ori_norm_data.shape[1], time_steps=dl.time_steps)
net = LSTM_Model(config)
net = net.cuda()
optimizer = torch.optim.Adam(net.parameters(),lr=lr)


In [ ]:
class My_Train_Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, output, target, info):
        print(type(output), type(target), type(info))
        return


In [ ]:
criterion_valid = nn.MSELoss(size_average=True).cuda()
criterion_train = My_Train_Loss().cuda()

In [ ]:
train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []
for epoch in range(epochs):
    net.train()
    for batch, (x, y_target) in enumerate(train_loader, 1):
        optimizer.zero_grad()
        x = x.cuda()
        y_target = y_target
        y_predict = net(x)
        start_index = batch * dl.batch_size + dl.time_steps - 1
        loss = criterion_train(y_predict, y_target, ori_data[start_index : start_index + dl.batch_size])
        train_losses.append(loss.item())

    if epoch % verify_every == 0:
        net.eval()
        y_predict = []
        # for i in range(valid_start, valid_end):




    loss = criterion_valid(y_predict[:, :3], y_target)


